In [8]:
from epsilon_transformers.process.processes import Mess3
from epsilon_transformers.process.MixedStateTree import MixedStateTree, MixedStateTreeNode
from epsilon_transformers.process.Process import _compute_emission_probabilities, _compute_next_distribution, Process
from collections import deque

In [11]:
mess3 = Mess3()
# msp_tree = mess3.derive_mixed_state_presentation(depth=11)


In [72]:
import numpy as np
def derive_mixed_state_presentation(process: Process, depth: int) -> MixedStateTree:
    starting_prob_vec = np.expand_dims(process.steady_state_vector, axis=0)
    tree_root = MixedStateTreeNode(state_prob_vector=starting_prob_vec, children=set(), path=[], emission_prob=0)
    nodes = set([tree_root])

    stack = deque([(tree_root, starting_prob_vec, [], 0)])
    while stack:
        current_node, state_prob_vector, current_path, current_depth = stack.pop()
        print(state_prob_vector.shape, current_path)
        if current_depth < depth:
            emission_probs = _compute_emission_probabilities(process, np.expand_dims(state_prob_vector[-1], axis=0))
            for emission in range(process.vocab_len):
                if emission_probs[emission] > 0:
                    next_state_prob_vector = _compute_next_distribution(process.transition_matrix, np.expand_dims(state_prob_vector[-1], axis=0), emission)
                    child_path = current_path + [emission]

                    full_state_prob_history = np.concatenate([state_prob_vector, np.expand_dims(next_state_prob_vector, axis=0)], axis=0)
                    print(state_prob_vector.shape, next_state_prob_vector.shape, child_path, full_state_prob_history.shape)

                    child_node = MixedStateTreeNode(
                        state_prob_vector=full_state_prob_history,
                        path=child_path,
                        children=set(),
                        emission_prob=emission_probs[emission]
                    )
                    current_node.add_child(child_node)

                    stack.append((child_node, next_state_prob_vector, child_path, current_depth + 1))
        nodes.add(current_node)
    
    return MixedStateTree(root_node=tree_root, process=process.name, nodes=nodes, depth=depth)

In [73]:
a = derive_mixed_state_presentation(mess3, 10)

(3,) []


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [23]:
tree_paths, tree_beliefs = a.paths_and_belief_states

In [25]:
tree_beliefs[0]

array([0.14467659, 0.51160179, 0.34372162, 0.12324287, 0.69474413,
       0.182013  ])